In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import libraryWAN as lw
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
sample_submission.csv
test.csv
train.csv



## ETL

In [70]:
from sklearn.model_selection import train_test_split

def read_data():
    df = pd.read_csv("input/train.csv")
    print ("Shape of base training File = ", df.shape)
    # Remove missing values and duplicates from training data
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df = read_data()
df_train, df_test = train_test_split(df, test_size = 0.02)
df_train = df_train.reset_index(drop=True)
df_train = df_train.reset_index()
df_train = df_train.rename(columns = {'index':'mat_idx'})

print (df_train.head(2))
print (df_test.shape)

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404288, 6)
   mat_idx      id    qid1    qid2  \
0        0  105401  173816  173817   
1        1  203850  306580  306581   

                                           question1  \
0  What are the top 5 online auction software com...   
1  What are the teachings of Siddhartha Gautama a...   

                                           question2  is_duplicate  
0        Online Auctions: How does BidHere.com work?             0  
1  What is Gautama Buddha famous for? What did he...             1  
(8086, 6)


## Load Pre-Acquired Function Words

In [71]:
data = pd.read_csv('function_words.csv',header=None,names=['fwords'])
fwords = data['fwords'].values.tolist()
fwords_dict = {}
for idx, word in enumerate(fwords):
    fwords_dict[word] = idx+1

## Helper Functions

In [72]:
transtable = {ord(c): None for c in string.punctuation}

def processSentence(raw_sentence):
    raw_list = raw_sentence.split(" ")
    clean_list = [str.translate(transtable) for str in raw_list] # remove punctuation
    clean_list = [word.lower().strip() for word in clean_list] # clean the word
    return clean_list

## Similarity Metrics

In [73]:
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler

minkowski_dis = DistanceMetric.get_metric('minkowski')

mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

features = np.zeros((df_train.shape[0], 5))

# Test run for the first 20 lines:
# for ridx, row in df_train.iterrows():
#     if ridx > 20: break
#     else:
        

In [74]:
s1 = df_train.get_value(0, 'question1')
s2 = df_train.get_value(0, 'question2')
s1_tok = processSentence(s1)
s2_tok = processSentence(s2)
WINDOW_SIZE = 10
ALPHA = 0.8

In [75]:
s1_wan = lw.normalizeWAN(lw.functionWordWAN(s1_tok, fwords_dict, WINDOW_SIZE, ALPHA), len(fwords))
s2_wan = lw.normalizeWAN(lw.functionWordWAN(s2_tok, fwords_dict, WINDOW_SIZE, ALPHA), len(fwords))

In [76]:
s1_wan

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.44444444,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [77]:
plt.imshow(s2_wan)
plt.show()

In [78]:
print(s1)
print(s2)
lw.relativeEntropy(s1_wan, s2_wan)

What are the top 5 online auction software companies?
Online Auctions: How does BidHere.com work?


[0.0, 0.0]

In [79]:
print(s1)

What are the top 5 online auction software companies?


In [80]:
print(s2)

Online Auctions: How does BidHere.com work?
